In [1]:
import pandas as pd
df1 = pd.read_pickle(r"Movie Dataframe with Ratings/networkData1.pkl")
df2 = pd.read_pickle(r"Movie Dataframe with Ratings/networkData2.pkl")
df3 = pd.read_pickle(r"Movie Dataframe with Ratings/networkData3.pkl")

frames = [df1, df2, df3]
df_reviews = pd.concat(frames, axis=0, ignore_index=True)
#df_reviews = pd.read_pickle(r'reviewsFromImdb/networkData3.pkl') #Read in the dataframe with reviews constructed from the "RetrievingMoviesAndReviews" notebook
df_reviews

,TitleId,MovieTitle,Actors,Genres,Reviews,Rating
0,0114709,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",[Andy's toys live a reasonable life of fun and...,8.3
1,0113497,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Among the thousands of films I have viewed, t...",7.0
2,0113228,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Sophia Loren, Bu...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",[Sequels are rarely half as good as the origin...,6.6
3,0114885,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[This was a good movie, even though I'm not th...",5.9
4,0113041,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Geo...","[{'id': 35, 'name': 'Comedy'}]","[As much as I want to rag this movie, make fun...",6.0
...,...,...,...,...,...,...
6089,0066579,Women in Love,"[Alan Bates, Oliver Reed, Glenda Jackson, Jenn...","[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",[Ken Russell's adaptation of D. H. Lawrence's ...,7.1
6090,0322824,Japón,"[Alejandro Ferretis, Magdalena Flores, Yolanda...","[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...","[Japon, a film by first-time director Carlos R...",6.9
6091,0285462,Boat Trip,"[Horatio Sanz, Roselyn Sanchez, Maurice Godin,...","[{'id': 35, 'name': 'Comedy'}]",[Man I don't understand why people come down s...,4.9
6092,0285531,Dreamcatcher,"[Morgan Freeman, Thomas Jane, Jason Lee, Damia...","[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...","[This Stephen King adaptation is underrated, s...",5.5


In [2]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+') #Create tokenizer with nltk library which only identifies words

In [3]:
#Reviews are split up into tokens which are words in order to be able to match tokens to the sentiment list

sent_tokens = []                                       #Initialize sentiment token list which holds reviews for a movie
for row in df_reviews["Reviews"]:                      #Run trough all rows in the review dataframe
    tokens = []                                        #Initialize token list which will hold each word of a review
    for sub in row:                                    #For each review in row
        tokens.append(tokenizer.tokenize(sub.lower())) #add tokens to list
    sent_tokens.append(tokens)                         #add list of tokens to list
df_reviews['Review_tokens'] = sent_tokens              #add list of lists to dataframe column


In [4]:
df_sentimentValues = pd.read_csv('Sentiment_values.txt', sep='\t',engine='python') #Read the sentiment dataframe
df_word_and_scores = df_sentimentValues[['word', 'happiness_average']].copy() #Extract interesting columns
sentiment_dict = dict(zip(df_word_and_scores.word, df_word_and_scores.happiness_average)) #Make dictionary - Keys = words, values = sentiment score

In [5]:
#Get the amount of sentiment eligible words for each review and store them in a list which will all be stored in 'ext_list' 

ext_list = []
for row in df_reviews['Review_tokens']:
    review_in_list =[]
    for reviews in row:
        tokens_in_list = []
        for token in reviews:
            if token in sentiment_dict.keys():
                tokens_in_list.append(token)
        review_in_list.append(len(tokens_in_list))
    ext_list.append(review_in_list)

In [6]:
#Similairly extract the sentiment score for each review and store them in a list which will all be store in the dataframe column 'Sentiment'

reviews_for_movie =[]
for row in df_reviews['Review_tokens']:
    sentiments_for_review = []
    for review in row:
        sentiments_for_token = []
        for token in review:
            if token in sentiment_dict.keys():
                sentiments_for_token.append(sentiment_dict[token])
        sentiments_for_review.append(int(sum(sentiments_for_token)))
    reviews_for_movie.append(sentiments_for_review)
df_reviews['Sentiment'] = reviews_for_movie

In [7]:
del ext_list[2455] #Remove troublesome index
df_reviews.drop(2455,inplace = True) #Remove troublesome row 
df_reviews = df_reviews.reset_index(drop=True) #Reset index of dataframe
del ext_list[5582] #Remove troublesome index
df_reviews.drop(5582,inplace = True) #Remove troublesome row 
df_reviews = df_reviews.reset_index(drop=True) #Reset index of dataframe
#del ext_list[5581] #Remove troublesome index
#df_reviews.drop(5581,inplace = True) #Remove troublesome row
df_reviews = df_reviews.reset_index(drop=True) #Reset index of dataframe

In [8]:
#Get average sentiment score of a movie by summing the sentiment score of a movie's reviews and dividing by the amount of sentiment eligible words in those reviews
import numpy as np
results = []
for x in range(len(df_reviews['Sentiment'])):
    print(all(item != 0 for item in df_reviews['Sentiment'][x]),x)
    results.append((sum([i / j for i, j in zip(df_reviews['Sentiment'][x], ext_list[x])]))/len(ext_list[x])) 
df_reviews['Avg sentiment movie'] = results #Average sentiment for movie is added as column

True 0
True 1
True 2
True 3
True 4
True 5
True 6
True 7
True 8
True 9
True 10
True 11
True 12
True 13
True 14
True 15
True 16
True 17
True 18
True 19
True 20
True 21
True 22
True 23
True 24
True 25
True 26
True 27
True 28
True 29
True 30
True 31
True 32
True 33
True 34
True 35
True 36
True 37
True 38
True 39
True 40
True 41
True 42
True 43
True 44
True 45
True 46
True 47
True 48
True 49
True 50
True 51
True 52
True 53
True 54
True 55
True 56
True 57
True 58
True 59
True 60
True 61
True 62
True 63
True 64
True 65
True 66
True 67
True 68
True 69
True 70
True 71
True 72
True 73
True 74
True 75
True 76
True 77
True 78
True 79
True 80
True 81
True 82
True 83
True 84
True 85
True 86
True 87
True 88
True 89
True 90
True 91
True 92
True 93
True 94
True 95
True 96
True 97
True 98
True 99
True 100
True 101
True 102
True 103
True 104
True 105
True 106
True 107
True 108
True 109
True 110
True 111
True 112
True 113
True 114
True 115
True 116
True 117
True 118
True 119
True 120
True 121
True 122
Tru

True 1309
True 1310
True 1311
True 1312
True 1313
True 1314
True 1315
True 1316
True 1317
True 1318
True 1319
True 1320
True 1321
True 1322
True 1323
True 1324
True 1325
True 1326
True 1327
True 1328
True 1329
True 1330
True 1331
True 1332
True 1333
True 1334
True 1335
True 1336
True 1337
True 1338
True 1339
True 1340
True 1341
True 1342
True 1343
True 1344
True 1345
True 1346
True 1347
True 1348
True 1349
True 1350
True 1351
True 1352
True 1353
True 1354
True 1355
True 1356
True 1357
True 1358
True 1359
True 1360
True 1361
True 1362
True 1363
True 1364
True 1365
True 1366
True 1367
True 1368
True 1369
True 1370
True 1371
True 1372
True 1373
True 1374
True 1375
True 1376
True 1377
True 1378
True 1379
True 1380
True 1381
True 1382
True 1383
True 1384
True 1385
True 1386
True 1387
True 1388
True 1389
True 1390
True 1391
True 1392
True 1393
True 1394
True 1395
True 1396
True 1397
True 1398
True 1399
True 1400
True 1401
True 1402
True 1403
True 1404
True 1405
True 1406
True 1407
True 1408


True 2216
True 2217
True 2218
True 2219
True 2220
True 2221
True 2222
True 2223
True 2224
True 2225
True 2226
True 2227
True 2228
True 2229
True 2230
True 2231
True 2232
True 2233
True 2234
True 2235
True 2236
True 2237
True 2238
True 2239
True 2240
True 2241
True 2242
True 2243
True 2244
True 2245
True 2246
True 2247
True 2248
True 2249
True 2250
True 2251
True 2252
True 2253
True 2254
True 2255
True 2256
True 2257
True 2258
True 2259
True 2260
True 2261
True 2262
True 2263
True 2264
True 2265
True 2266
True 2267
True 2268
True 2269
True 2270
True 2271
True 2272
True 2273
True 2274
True 2275
True 2276
True 2277
True 2278
True 2279
True 2280
True 2281
True 2282
True 2283
True 2284
True 2285
True 2286
True 2287
True 2288
True 2289
True 2290
True 2291
True 2292
True 2293
True 2294
True 2295
True 2296
True 2297
True 2298
True 2299
True 2300
True 2301
True 2302
True 2303
True 2304
True 2305
True 2306
True 2307
True 2308
True 2309
True 2310
True 2311
True 2312
True 2313
True 2314
True 2315


True 3465
True 3466
True 3467
True 3468
True 3469
True 3470
True 3471
True 3472
True 3473
True 3474
True 3475
True 3476
True 3477
True 3478
True 3479
True 3480
True 3481
True 3482
True 3483
True 3484
True 3485
True 3486
True 3487
True 3488
True 3489
True 3490
True 3491
True 3492
True 3493
True 3494
True 3495
True 3496
True 3497
True 3498
True 3499
True 3500
True 3501
True 3502
True 3503
True 3504
True 3505
True 3506
True 3507
True 3508
True 3509
True 3510
True 3511
True 3512
True 3513
True 3514
True 3515
True 3516
True 3517
True 3518
True 3519
True 3520
True 3521
True 3522
True 3523
True 3524
True 3525
True 3526
True 3527
True 3528
True 3529
True 3530
True 3531
True 3532
True 3533
True 3534
True 3535
True 3536
True 3537
True 3538
True 3539
True 3540
True 3541
True 3542
True 3543
True 3544
True 3545
True 3546
True 3547
True 3548
True 3549
True 3550
True 3551
True 3552
True 3553
True 3554
True 3555
True 3556
True 3557
True 3558
True 3559
True 3560
True 3561
True 3562
True 3563
True 3564


True 4715
True 4716
True 4717
True 4718
True 4719
True 4720
True 4721
True 4722
True 4723
True 4724
True 4725
True 4726
True 4727
True 4728
True 4729
True 4730
True 4731
True 4732
True 4733
True 4734
True 4735
True 4736
True 4737
True 4738
True 4739
True 4740
True 4741
True 4742
True 4743
True 4744
True 4745
True 4746
True 4747
True 4748
True 4749
True 4750
True 4751
True 4752
True 4753
True 4754
True 4755
True 4756
True 4757
True 4758
True 4759
True 4760
True 4761
True 4762
True 4763
True 4764
True 4765
True 4766
True 4767
True 4768
True 4769
True 4770
True 4771
True 4772
True 4773
True 4774
True 4775
True 4776
True 4777
True 4778
True 4779
True 4780
True 4781
True 4782
True 4783
True 4784
True 4785
True 4786
True 4787
True 4788
True 4789
True 4790
True 4791
True 4792
True 4793
True 4794
True 4795
True 4796
True 4797
True 4798
True 4799
True 4800
True 4801
True 4802
True 4803
True 4804
True 4805
True 4806
True 4807
True 4808
True 4809
True 4810
True 4811
True 4812
True 4813
True 4814


In [9]:
df_reviews.to_pickle("Sentiment_for_reviews.pkl") #Export dataframe
df_reviews

,TitleId,MovieTitle,Actors,Genres,Reviews,Rating,Review_tokens,Sentiment,Avg sentiment movie
0,0114709,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",[Andy's toys live a reasonable life of fun and...,8.3,"[[andy, s, toys, live, a, reasonable, life, of...","[2610, 868, 1292, 4159, 885, 455, 4124, 515, 3...",5.566495
1,0113497,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Among the thousands of films I have viewed, t...",7.0,"[[among, the, thousands, of, films, i, have, v...","[1520, 3032, 1438, 652, 1035, 759, 706, 1141, ...",5.521536
2,0113228,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Sophia Loren, Bu...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",[Sequels are rarely half as good as the origin...,6.6,"[[sequels, are, rarely, half, as, good, as, th...","[3242, 1686, 939, 742, 987, 1108, 172, 745, 92...",5.533041
3,0114885,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[This was a good movie, even though I'm not th...",5.9,"[[this, was, a, good, movie, even, though, i, ...","[804, 1084, 1067, 1824, 857, 3986, 1290, 751, ...",5.424351
4,0113041,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Geo...","[{'id': 35, 'name': 'Comedy'}]","[As much as I want to rag this movie, make fun...",6.0,"[[as, much, as, i, want, to, rag, this, movie,...","[796, 1196, 601, 287, 780, 842, 1028, 1469, 10...",5.599556
...,...,...,...,...,...,...,...,...,...
6087,0066579,Women in Love,"[Alan Bates, Oliver Reed, Glenda Jackson, Jenn...","[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",[Ken Russell's adaptation of D. H. Lawrence's ...,7.1,"[[ken, russell, s, adaptation, of, d, h, lawre...","[1930, 1377, 3407, 2049, 1188, 423, 576, 825, ...",5.494530
6088,0322824,Japón,"[Alejandro Ferretis, Magdalena Flores, Yolanda...","[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...","[Japon, a film by first-time director Carlos R...",6.9,"[[japon, a, film, by, first, time, director, c...","[2763, 2241, 1864, 493, 1629, 1440, 794, 832, ...",5.402044
6089,0285462,Boat Trip,"[Horatio Sanz, Roselyn Sanchez, Maurice Godin,...","[{'id': 35, 'name': 'Comedy'}]",[Man I don't understand why people come down s...,4.9,"[[man, i, don, t, understand, why, people, com...","[965, 539, 600, 2531, 312, 961, 641, 772, 5041...",5.492860
6090,0285531,Dreamcatcher,"[Morgan Freeman, Thomas Jane, Jason Lee, Damia...","[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...","[This Stephen King adaptation is underrated, s...",5.5,"[[this, stephen, king, adaptation, is, underra...","[663, 3448, 1905, 4429, 2080, 555, 1621, 1369,...",5.391698


In [14]:
#Showcase of movie sentiment extremes
print(min(df_reviews['Avg sentiment movie']))
print(max(df_reviews['Avg sentiment movie']))

5.184782608695652
5.721900826446281
